In [ ]:
# !pip install face_recognition
# !pip install opencv-python

In [ ]:
import cv2  # For computer vision tasks, such as capturing and processing images and video
import face_recognition  # For face detection and face recognition
import os  # For interacting with the operating system, such as file and directory management
import pickle  # For saving and loading Python objects like models and data

<h2 style='color:blue' align='center'>Function to register a new member</h2>

In [ ]:
import cv2
import face_recognition
import os
import pickle

def register_new_member(name, age, id_number):
    # Create a folder for the new member
    member_folder = f"members/{name}"
    os.makedirs(member_folder, exist_ok=True)
    
    # Save the member details
    with open(os.path.join(member_folder, "details.txt"), "w") as f:
        f.write(f"Name: {name}\nAge: {age}\nID: {id_number}\n")
    
    # Capture the face
    cap = cv2.VideoCapture(0)
    print("Capturing face... Press 's' to capture")
    
    while True:
        ret, frame = cap.read()
        cv2.imshow("Capture Face", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('s'):
            face_locations = face_recognition.face_locations(frame)
            
            if face_locations:
                face_encoding = face_recognition.face_encodings(frame, face_locations)[0]
                
                # Save the face encoding
                with open(os.path.join(member_folder, "face_encoding.pkl"), "wb") as f:
                    pickle.dump(face_encoding, f)
                
                print("Face captured and details saved.")
                break
            else:
                print("No face detected. Try again.")
    
    cap.release()
    cv2.destroyAllWindows()

# Input from users
name = input("Enter name: ")
age = input("Enter age: ")
id_number = input("Enter ID number: ")
register_new_member(name, age, id_number)


<h2 style='color:blue' align='center'>Function to detected and identify face</h2>

In [ ]:
import cv2
import face_recognition
import os
import pickle

# Function to recognize and verify faces
def recognize_faces():
    known_faces = []
    known_details = []

    # Load all face encodings and details
    for member in os.listdir("members"):
        member_folder = os.path.join("members", member)
        if os.path.isdir(member_folder):
            encoding_path = os.path.join(member_folder, "face_encoding.pkl")
            details_path = os.path.join(member_folder, "details.txt")
            if os.path.exists(encoding_path) and os.path.exists(details_path):
                with open(encoding_path, "rb") as f:
                    known_faces.append(pickle.load(f))
                with open(details_path, "r") as f:
                    known_details.append(f.read())

    # Capture video
    cap = cv2.VideoCapture(0)
    print("Detecting face... Press 'q' to quit")

    while True:
        ret, frame = cap.read()
        face_locations = face_recognition.face_locations(frame)
        face_encodings = face_recognition.face_encodings(frame, face_locations)

        for face_encoding, face_location in zip(face_encodings, face_locations):
            matches = face_recognition.compare_faces(known_faces, face_encoding)
            name = "Unknown"
            details = ""

            if True in matches:
                match_index = matches.index(True)
                name = known_details[match_index].split('\n')[0].split(': ')[1]
                details = known_details[match_index]

            top, right, bottom, left = face_location
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.putText(frame, name, (left + 6, bottom + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            cv2.putText(frame, details, (left + 6, bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        cv2.imshow("Face Recognition", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

recognize_faces()
